<a href="https://colab.research.google.com/github/pankaj18/NLP-Projects/blob/master/nlp_preprocessing/Keras_LSTM_IMDB_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.

#### Notes
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc

In [1]:
#Import libraries
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb #in-built imdb dataset available in keras

max_features = 20000
maxlength = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 36

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features) #loading dataset here
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlength)
x_test = sequence.pad_sequences(x_test, maxlen=maxlength)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Epoch 1/20
695/695 [==============================] - 222s 312ms/step - loss: 0.5367 - accuracy: 0.7089 - val_loss: 0.3529 - val_accuracy: 0.8437
Epoch 2/20
695/695 [==============================] - 215s 310ms/step - loss: 0.2488 - accuracy: 0.9017 - val_loss: 0.4732 - val_accuracy: 0.8150
Epoch 3/20
695/695 [==============================] - 214s 308ms/step - loss: 0.1636 - accuracy: 0.9387 - val_loss: 0.4381 - val_accuracy: 0.8294
Epoch 4/20
695/695 [==============================] - 215s 309ms/step - loss: 0.0959 - accuracy: 0.9667 - val_loss: 0.6071 - val_accuracy: 0.8106
Epoch 5/20
695/695 [==============================] - 214s 308ms/step - loss: 0.0672 - accuracy: 0.9766 - val_loss: 0.6360 - val_accuracy: 0.8220
Epoch 6/20
695/695 [==============================] - 213s 307ms/step - loss: 0.0473 - accuracy: 0.9844 - val_loss: 0.6

Explain the model

In [2]:
!pip install shap
import shap

# we are taking first 150 training dataset as our background dataset to integerate over it
explainer = shap.DeepExplainer(model, x_train[:150])

# explain the first 20 predictions
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(x_test[:20])

     |████████████████████████████████| 358kB 7.4MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491641 sha256=3398a4028d36bfe50cb69cd2a9c2334c04e2285f1c37f450cba99196520f3e11
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap


keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


TypeError: ignored

In [ ]:
# init JS visualization code
shap.initjs()

# index to words transforming
import numpy as np
words = imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])

# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_words[0])

Here, We note that each sample is an IMDB review text document, represented as the sequence of words. This means that "feature 0" is the first word in the review, which will be different for difference reviews. This means calling summary_plot will combine the importance of all the words by their position in the text. This is likely not what you want for a global measure of feature importance (which is why we have not called summary_plot here). If you do want a global summary of a word's importance you could pull apart the feature attribution values and group them by words.